In [140]:
import pandas as pd
import numpy as np
import datetime
from openpyxl import load_workbook

In [185]:
SEASON = 12
MATCHDAY = 5

In [186]:
mc = pd.read_excel('data/match_'+str(MATCHDAY)+'.xlsx')
mc.drop('№', axis=1, inplace=True)
mc.rename(columns={'index': 'Name'}, inplace=True)

In [187]:
mc.columns

Index(['Name', 'Итог', 'атлетико – челси', '0\n1', 'лацио – бавария', '1\n4',
       'аталанта – реал мадрид', '0\n1.1', 'боруссия м. – манчестер сити',
       '0\n2', 'арсенал – бенфика', '3\n2', 'аякс – лилль', '2\n1',
       'вильярреал – зальцбург', '2\n1.1', 'динамо загреб – краснодар', '1\n0',
       'брюгге – динамо киев', '0\n1.2', 'псв – олимпиакос', '2\n1.2'],
      dtype='object')

In [188]:
#Заголовки нулевой таблицы для создания ДБ ставок
zeros_col = ['Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']

matches = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='matches', index_col=0)
players = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='players', index_col=0)
season = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='season', index_col=0)
teams = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='teams', index_col=0)

In [189]:
def proceed_bet(player_id, match_id):
    """
    proceed_bet(player_id, match_id)
    Сверяет ставки с результатом матча.
    player_id = Id участника
    match_id = Id матча
    """
    player_name = players['Name'].iloc[player_id]
    
    h_team = matches['Hometeam'].loc[match_id]
    a_team = matches['Awayteam'].loc[match_id]
    match = str(h_team)+' - '+str(a_team)
    
    try:
        hscore_true = matches['H_score'].loc[match_id]
        ascore_true = matches['A_score'].loc[match_id]

        hscore_bet = int(bets['bet_hscore_'+str(match_id)].loc[player_id])
        ascore_bet = int(bets['bet_ascore_'+str(match_id)].loc[player_id])
        
    except ValueError:
        print(player_name, 'made no bet on', match)
        bets['points_'+str(match_id)].loc[player_id] = 0
        return
    
    true_score= str(hscore_true)+':'+str(ascore_true)
    bet_score= str(hscore_bet)+':'+str(ascore_bet)
    
    points_granted = 0
    
    if (hscore_true == hscore_bet and ascore_true == ascore_bet):
        points_granted = 5
        matches['Exact Score'].loc[match_id] += 1
        season['Exact Score'].loc[player_id] += 1
        bets['Exact Score'].loc[player_id] += 1
    elif (hscore_true - ascore_true) == (hscore_bet - ascore_bet):
        points_granted = 3
        matches['Right Margin'].loc[match_id] += 1
        season['Right Margin'].loc[player_id] += 1
        bets['Right Margin'].loc[player_id] += 1
    elif ((hscore_true > ascore_true) and (hscore_bet > ascore_bet)) or ((hscore_true < ascore_true) and (hscore_bet < ascore_bet)):
        points_granted = 2
        matches['Right Result'].loc[match_id] += 1
        season['Right Result'].loc[player_id] += 1
        bets['Right Result'].loc[player_id] += 1
    else:
        matches['Wrong Result'].loc[match_id] += 1
        season['Wrong Result'].loc[player_id] += 1
        bets['Wrong Result'].loc[player_id] += 1
    matches['Bets'].loc[match_id] += 1
    season['Bets'].loc[player_id] += 1
    bets['points_'+str(match_id)].loc[player_id] = points_granted
    return

def count_total(player_id):
    """
    Считает суммарное число баллов, суммируя значения с bets 
    """
    total = 0
    for i in range(11, bets.shape[1], 3):
        total += bets.loc[player_id].iloc[i]
    bets['Game Points'].loc[player_id] = total
    season['Game Points'].loc[player_id] += total
    print(bets.iloc[player_id, 0], total)
    
def find_uniques(match_ids):
    h_score = str(matches['H_score'].loc[match_ids])
    a_score = str(matches['A_score'].loc[match_ids])
    player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
    bets['Uniques'].loc[player_id] += 1
    season['Uniques'].loc[player_id] += 1
    matches['Uniques'].iloc[match_ids] += 1
    return player_id

def get_match_ids(season=SEASON, matchday=MATCHDAY):
    ids = matches[matches['Season'] == season][matches['Matchday'] == matchday].index
    return_list = ['Name', 'Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']
    for i in ids:
        return_list.append('bet_hscore_'+str(i))
        return_list.append('bet_ascore_'+str(i))
        return_list.append('points_'+str(i))
    return return_list

In [190]:
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]
      
    if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
        print(f'{hometeam} - {awayteam} уже есть')
    else:
        matches = matches.append({'Hometeam': hometeam, 'Awayteam': awayteam,
                                  'H_score': int(h_score), 'A_score': int(a_score),
                                  'Season': SEASON, 'Matchday': MATCHDAY, 'Date': datetime.datetime.now(), 'Bets': 0,
                                  'Exact Score': 0, 'Right Margin': 0, 'Right Result': 0, 
                                  'Wrong Result': 0, 'Uniques': 0}, ignore_index=True)
    
bets_columns = get_match_ids(SEASON,MATCHDAY)
bets = pd.DataFrame(pd.concat([mc['Name'], pd.DataFrame(np.zeros((mc['Name'].shape[0],8)), columns=zeros_col)], axis=1), columns=bets_columns)  

#Очень тупо, я повторяю цикл, потому что между этими операциями ме надо добавить БД для ставок
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]    
    match_id = matches[matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].index[0]   
    
    bets['bet_hscore_'+str(match_id)] = mc[match].str.split(':').str[0]
    bets['bet_ascore_'+str(match_id)] = mc[match].str.split(':').str[1]

<ipython-input-190-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-190-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-190-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-190-7dd6f4ee2aab>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
<ipython-input-190-7dd6f4ee2aab>:5: UserWarning:

In [191]:
for player in range(0,players['Name'].shape[0]):
    for match in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
        proceed_bet(player,match)

H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [192]:
mc

,Name,Итог,атлетико – челси,0\n1,лацио – бавария,1\n4,аталанта – реал мадрид,0\n1.1,боруссия м. – манчестер сити,0\n2,...,аякс – лилль,2\n1,вильярреал – зальцбург,2\n1.1,динамо загреб – краснодар,1\n0,брюгге – динамо киев,0\n1.2,псв – олимпиакос,2\n1.2
0,Алферов Ян,23,1:0,0,0:3,3,1:1,0,0:2,5,...,1:1,0,2:1,5,3:1,2,1:0,0,2:1,5
1,Андреев Иван,14,1:1,0,1:4,6,1:2,3,1:3,3,...,1:1,0,1:1,0,1:2,0,2:0,0,1:2,0
2,Афанасьев Владимир,8,2:1,0,1:1,0,2:1,0,1:2,2,...,2:0,2,1:1,0,2:0,2,2:1,0,1:1,0
3,Белькин Сергей,14,2:1,0,1:2,2,2:0,0,0:3,2,...,3:1,2,1:1,0,1:1,0,1:1,0,2:1,5
4,Гришин Антон,20,2:1,0,1:2,2,2:1,0,1:2,2,...,1:1,0,2:1,5,2:1,3,1:1,0,2:1,5
5,Данилов Евгений,11,2:0,0,0:4,2,3:1,0,1:4,2,...,2:2,0,0:2,0,2:0,2,1:1,0,2:1,5
6,Еременко Владислав,29,1:2,3,1:3,2,2:4,2,1:2,2,...,1:1,0,2:1,5,2:0,2,0:1,8,2:0,2
7,Жигалов Александр,20,1:0,0,1:3,2,2:1,0,0:1,2,...,1:0,3,1:0,3,1:0,5,1:0,0,3:1,2
8,Зубатов Михаил,18,0:0,0,1:2,2,3:1,0,0:2,5,...,2:1,5,1:2,0,2:1,3,2:1,0,1:1,0
9,Колодин Дмитрий,20,0:0,0,1:3,2,1:3,2,2:1,0,...,2:1,5,2:3,0,2:1,3,2:1,0,2:1,5


In [193]:
bets['Name']

0             Алферов Ян
1           Андреев Иван
2     Афанасьев Владимир
3         Белькин Сергей
4           Гришин Антон
5        Данилов Евгений
6     Еременко Владислав
7      Жигалов Александр
8         Зубатов Михаил
9        Колодин Дмитрий
10        Конаков Никита
11       Котов Александр
12    Криеванс Владислав
13      Кучеренко Никита
14      Лесик Константин
15          Малевич Егор
16       Махмудов Руслан
17           Минеев Иван
18         Муратов Игорь
19        Невский Леонид
20     Новиков Александр
21        Оксанич Кирилл
22     Сибиряков Георгий
23       Титаренко Антон
24      Фёдоров Владимир
25     Чапаров Александр
26          Шевчук Антон
Name: Name, dtype: object

In [194]:
def find_match_id(tour):
    return 11+(tour-1)*3
find_match_id(9)

35

In [196]:
bets.iloc[6,35] = 7


In [197]:
for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
    player_id = find_uniques(i)
    bets['points_'+str(i)].loc[player_id] += 1
    
for player in range(0,players['Name'].shape[0]):
    count_total(player)

Алферов Ян 23.0
Андреев Иван 14.0
Афанасьев Владимир 8.0
Белькин Сергей 14.0
Гришин Антон 20.0
Данилов Евгений 11.0
Еременко Владислав 29.0
Жигалов Александр 20.0
Зубатов Михаил 18.0
Колодин Дмитрий 20.0
Конаков Никита 24.0
Котов Александр 22.0
Криеванс Владислав 17.0
Кучеренко Никита 17.0
Лесик Константин 23.0
Малевич Егор 23.0
Махмудов Руслан 21.0
Минеев Иван 18.0
Муратов Игорь 21.0
Невский Леонид 23.0
Новиков Александр 11.0
Оксанич Кирилл 27.0
Сибиряков Георгий 19.0
Титаренко Антон 14.0
Фёдоров Владимир 18.0
Чапаров Александр 16.0
Шевчук Антон 12.0


<ipython-input-197-c524e9ee286a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
<ipython-input-189-0ecd95017322>:69: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
H:\dev\py\python-3.8.5.amd64\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


### Bombers

In [198]:
bombs = pd.read_excel('output/bombs_bets.xlsx', index_col=0, sheet_name=str(MATCHDAY))


In [199]:
# Сгрупировал таблицу в формате имя - очки и присвоил в новую переменную
bombs_points = bombs[bombs['Points']>0].groupby(['Name']).sum().sort_values(by=['Points'], ascending=False)

# Теперь итерируем по всей таблице bets. Если имя есть в bomb_points обновляем значение таблицы Bomb Points. 
# Важно вызывать через loc чтобы не вызывать копию, а работать напрямую с табилцей
print('Очки полученные за бомбардиров')
for row in np.arange(bets.shape[0]):
    try:
        bets.loc[row,'Bomb Points'] = bombs_points['Points'][bets.loc[row,'Name']]
        print(bets.loc[row,'Bomb Points'], bets.loc[row,'Name'])
    except KeyError:
        pass

Очки полученные за бомбардиров
6.0 Алферов Ян
2.0 Андреев Иван
4.0 Афанасьев Владимир
2.0 Гришин Антон
2.0 Данилов Евгений
6.0 Еременко Владислав
4.0 Жигалов Александр
6.0 Зубатов Михаил
2.0 Колодин Дмитрий
8.0 Конаков Никита
2.0 Котов Александр
2.0 Криеванс Владислав
6.0 Кучеренко Никита
4.0 Лесик Константин
2.0 Малевич Егор
4.0 Минеев Иван
2.0 Муратов Игорь
2.0 Невский Леонид
4.0 Оксанич Кирилл
6.0 Сибиряков Георгий
2.0 Титаренко Антон
6.0 Фёдоров Владимир


In [200]:
bets['Total Points'] = bets['Game Points'] + bets['Bomb Points']

season['Bomb Points'] += bets['Bomb Points']
season['Total Points'] = season['Game Points'] + season['Bomb Points']

In [201]:
bets.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
6,Еременко Владислав,35.0
10,Конаков Никита,32.0
21,Оксанич Кирилл,31.0
0,Алферов Ян,29.0
14,Лесик Константин,27.0
22,Сибиряков Георгий,25.0
19,Невский Леонид,25.0
15,Малевич Егор,25.0
24,Фёдоров Владимир,24.0
7,Жигалов Александр,24.0


In [202]:
season.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
0,Алферов Ян,93.0
14,Лесик Константин,91.0
21,Оксанич Кирилл,90.0
4,Гришин Антон,90.0
19,Невский Леонид,85.0
7,Жигалов Александр,83.0
16,Махмудов Руслан,77.0
11,Котов Александр,76.0
15,Малевич Егор,76.0
18,Муратов Игорь,75.0


In [203]:
for match_id in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
    match = matches.iloc[match_id,:]
    hometeam, awayteam = match[0], match[1]
    m_s = matches.iloc[match_id,:]    #Match_series
    for team in [hometeam, awayteam]:
        # Какое говно)))))
        if team == m_s['Hometeam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 1,0,0
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 0,0,1
            else:
                win, draw, loss = 0,1,0
        elif team == m_s['Awayteam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 0,0,1
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 1,0,0
            else:
                win, draw, loss = 0,1,0
                      
        if team in teams['Team'].values:
            team_index = teams[teams['Team'] == team].index[0]
            teams.iloc[team_index,2] += 1
            teams.iloc[team_index,3] += win
            teams.iloc[team_index,4] += draw
            teams.iloc[team_index,5] += loss
            teams.iloc[team_index,6] += m_s['Bets']
            teams.iloc[team_index,7] += m_s['Exact Score']
            teams.iloc[team_index,8] += m_s['Right Margin']
            teams.iloc[team_index,9] += m_s['Right Result']
            teams.iloc[team_index,10] += m_s['Wrong Result']
            teams.iloc[team_index,11] += m_s['Uniques']
        else:
            teams = teams.append({'Team': team, 'Matches': 1, 'Wins': win, 'Draws': draw, 'Losses': loss,
                                 'Bets': m_s['Bets'], 'Exact Score': m_s['Exact Score'], 'Right Margin': m_s['Right Margin'], 
                                 'Right Result': m_s['Right Result'], 'Wrong Result': m_s['Wrong Result'], 
                                 'Uniques': m_s['Uniques']}, ignore_index=True)

In [204]:
teams

,Team,Country,Matches,Wins,Draws,Losses,Bets,Exact Score,Right Margin,Right Result,Wrong Result,Uniques
0,леванте,NaN,1,0,1,0,25,1,5,0,19,1
1,гранада,NaN,1,0,1,0,25,1,5,0,19,1
2,майнц,NaN,1,1,0,0,27,1,2,1,23,1
3,унион берлин,NaN,2,1,0,1,50,1,3,1,45,1
4,ньюкасл юнайтед,NaN,1,1,0,0,27,0,4,3,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...
66,бавария,NaN,1,1,0,0,27,1,1,21,4,1
67,реал мадрид,NaN,1,1,0,0,27,1,6,2,18,1
68,зальцбург,NaN,1,0,0,1,27,7,4,3,13,0
69,псв,NaN,1,1,0,0,27,11,2,5,9,0


In [205]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
book = load_workbook('output/bets.xlsx')
writer = pd.ExcelWriter('output/bets.xlsx', engine='openpyxl')
writer.book = book

# Write each dataframe to a different worksheet.
bets.to_excel(writer, sheet_name=str(MATCHDAY))

writer.save()
writer.close()

In [206]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'output/season_{str(SEASON)}.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
season.to_excel(writer, sheet_name='season')
matches.to_excel(writer, sheet_name='matches')
players.to_excel(writer, sheet_name='players')
teams.to_excel(writer, sheet_name='teams')
#scorers.to_excel(writer, sheet_name='scorers_18')

writer.save()